In [1]:
import os,sys,pickle,random,threading,time
import platform,subprocess,glob,datetime
import numpy as np
import cv2
from PyQt5 import QtCore
from PyQt5 import QtWidgets

# this is needed to run the Qt5 loop in the background 
# see http://localhost:8888/notebooks/VisibleMale.ipynb#
%gui qt
%matplotlib qt
print([it for it in sys.modules.keys() if "pyqt" in it.lower()])

# hardcoded path (CHANGE AS NEEDED)
sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")

from OpenVisus                        import *
from OpenVisus.gui                    import *
from OpenVisus.image_utils            import *

['PyQt5', 'PyQt5.sip', 'PyQt5.QtCore', 'PyQt5.QtGui', 'PyQt5.QtWidgets']
Starting OpenVisus C:\projects\OpenVisus\build\RelWithDebInfo\OpenVisus\__init__.py 3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)] sys.version_info(major=3, minor=8, micro=6, releaselevel='final', serial=0) ...
QT_PLUGIN_PATH c:\python38\lib\site-packages\PyQt5\Qt/plugins


In [2]:
# do not change this cell for very good reasons (otherwise Qt5 loop won't work)
import time
time.sleep(2)

# Example: open a dataset or a scene

In [3]:
# NOTE: no need to run the viewer (QT main loop is already running in the background)
scene_filename=r"D:\GoogleSci\visus_dataset\2kbit1\zip\rowmajor\visus.idx"
viewer=PyViewer()
viewer.open(scene_filename)
viewer.showMaximized()

2021-10-16 18:45:47.204576 PyScriptingNode Got in input (128, 128, 256) uint8 origin Volume 
2021-10-16 18:45:47.210579 PyScriptingNode Output is  (128, 128, 256) uint8 msec 5 


# Utilities to run actions in the background

In [266]:
# //////////////////////////////////////////////////////////////
class RunActions:
    
    def __init__(self):
        self.actions=[]
        self.cursor=0
        self.timer=QtCore.QTimer()
        self.timer.timeout.connect(self.onTimer)
        
    def append(self, msec, fn,**kwargs):
        self.actions.append([msec,lambda : fn(**kwargs)])
        
    def onTimer(self):
        fn=self.actions[self.cursor][1]
        fn()
        self.cursor+=1
        self.timer.stop()
        if self.cursor<len(self.actions): 
            msec=self.actions[self.cursor][0]
            self.timer.start(msec) 
            
    def start(self):
        if self.actions:
            msec=self.actions[0][0]
            self.timer.start(msec)      
        
    def stop(self):
        self.timer.stop()
        
def RotateScene(axis=(0,0,1),angle=10):
    print("RotateScene",axis,angle)
    glcamera=viewer.getGLCamera()
    viewer.getGLCamera().setRotation(glcamera.getRotation() * Quaternion(Point3d(axis),math.radians(angle)))
    viewer.refreshAll()
    viewer.postRedisplay()
    
def TakeSnapshot(filename="temp.png",):
    print("TakeSnapshotAction",filename)  
    if False:
        viewer.takeSnapshot(False,filename) 
    else:
        viewer_py=sip.wrapinstance(FromCppQtWidget(viewer.c_ptr()), QMainWindow)
        screenshot = QtWidgets.QApplication.primaryScreen().grabWindow(viewer_py.winId() )
        screenshot.save(filename)
    
def OpenScene(filename=""):
    print("OpenScene",filename)
    viewer.open(filename)
    
def CreateViewer():
    viewer=PyViewer()
    viewer_py=sip.wrapinstance(FromCppQtWidget(viewer.c_ptr()), QMainWindow)
    viewer_py.setVisible(True)
    viewer_py.show()
    viewer_py.setFocus()
    viewer_py.showMaximized()
    viewer_py.activateWindow()
    return viewer

# Open a dataset and rotate the scene

In [268]:
# important to create the viewer in a different cell otherwise it won't get the focus
viewer=CreateViewer()

In [270]:
viewer.open(r"D:\GoogleSci\visus_dataset\2kbit1\zip\rowmajor\visus.idx")

# no selection
viewer.dropSelection()

# **** make sure that the dataset node has UUID `dataset` (you can save an xml and inspect it) ***
dataset_node=DatasetNode.castFrom(viewer.findNodeByUUID("dataset"))
dataset_node.setShowBounds(False)

# run actions inside a PyQt5 timer (otherwise they won't work)
actions=RunActions()
for I in range(10):
    actions.append(200,RotateScene,axis=(0,0,1),angle=10)
    actions.append(  0,TakeSnapshot,filename=r"D:\bbb-{:04d}.png".format(I))
    
actions.start()

2021-06-01 14:09:50.591926 PyScriptingNode Output is  (128, 128, 256) uint8 msec 2 
RotateScene (0, 0, 1) 10
TakeSnapshotAction D:\bbb-0000.png
2021-06-01 14:10:01.148585 PyScriptingNode Got in input (128, 128, 256) uint8 origin Volume 
2021-06-01 14:10:01.150573 PyScriptingNode Output is  (128, 128, 256) uint8 msec 2 
2021-06-01 14:10:01.180585 PyScriptingNode Got in input (128, 128, 256) uint8 origin Volume 
2021-06-01 14:10:01.182585 PyScriptingNode Output is  (128, 128, 256) uint8 msec 2 
RotateScene (0, 0, 1) 10
TakeSnapshotAction D:\bbb-0001.png
2021-06-01 14:10:01.483586 PyScriptingNode Got in input (64, 64, 128) uint8 origin Volume 
2021-06-01 14:10:01.485573 PyScriptingNode Output is  (64, 64, 128) uint8 msec 2 
2021-06-01 14:10:01.548572 PyScriptingNode Got in input (128, 256, 256) uint8 origin Volume 
2021-06-01 14:10:01.550578 PyScriptingNode Output is  (128, 256, 256) uint8 msec 1 
RotateScene (0, 0, 1) 10
TakeSnapshotAction D:\bbb-0002.png
2021-06-01 14:10:01.801666 PyScr

# Open Viewer snapshots and play them

In [271]:
# important to create the viewer in a different cell otherwise it won't get the focus
viewer=CreateViewer()

2021-06-01 14:10:04.049606 PyScriptingNode Output is  (128, 128, 256) uint8 msec 2 


In [272]:
snapshots=sorted(glob.glob(r"C:\projects\OpenVisus\Samples\jupyter\honeycomb-demo1\visusviewer.snapshot*xml"))
snapshots=snapshots[:5] 

actions=RunActions()
for I,snapshot in enumerate(snapshots):
    print("\t",I,snapshot)
    actions.append(200,OpenScene,filename=snapshot)
    actions.append(  0,TakeSnapshot,filename=r"D:\aaa-{:04d}.png".format(I))
actions.start()

	 0 C:\projects\OpenVisus\Samples\jupyter\honeycomb-demo1\visusviewer.snapshot.20210531145607684.xml
	 1 C:\projects\OpenVisus\Samples\jupyter\honeycomb-demo1\visusviewer.snapshot.20210531145641394.xml
	 2 C:\projects\OpenVisus\Samples\jupyter\honeycomb-demo1\visusviewer.snapshot.20210531145646426.xml
	 3 C:\projects\OpenVisus\Samples\jupyter\honeycomb-demo1\visusviewer.snapshot.20210531145715830.xml
	 4 C:\projects\OpenVisus\Samples\jupyter\honeycomb-demo1\visusviewer.snapshot.20210531145847913.xml
OpenScene C:\projects\OpenVisus\Samples\jupyter\honeycomb-demo1\visusviewer.snapshot.20210531145607684.xml
TakeSnapshotAction D:\aaa-0000.png
2021-06-01 14:10:20.408598 PyScriptingNode Got in input (347, 98, 704) float32 origin Volume 
2021-06-01 14:10:20.410600 PyScriptingNode Output is  (347, 98, 704) float32 msec 2 
OpenScene C:\projects\OpenVisus\Samples\jupyter\honeycomb-demo1\visusviewer.snapshot.20210531145641394.xml
TakeSnapshotAction D:\aaa-0001.png
2021-06-01 14:10:20.978601 PyScr